# Extended DMD with control for nonlinear systems
We apply extended dynamic mode decomposition for control (EDMDc) to a low-dimensional, nonlinear system
(this is example in Sec. 4 in Proctor et al., _"Dynamic Mode Decomposition with Control"_, SIAM 2016):

$$x_{k+1} =\begin{bmatrix} 1.5 & 0\\ 0 & 0.1 \end{bmatrix} + \begin{bmatrix} 1\\ 0 \end{bmatrix} u$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

import pykoopman as pk

Integrate system

In [ ]:
A = np.matrix([[1.5, 0],[0, 0.1]])
B = np.matrix([[1],[0]])

def advance_linear_system(x0,u,n,A=A,B=B):
    # A = np.matrix([[1.5, 0],[0, 0.1]])
    # B = np.matrix([1,0])
    x = np.zeros([n,len(x0)])
    x[0,:] = x0
    for i in range(n-1):
        x[i+1,:] = A.dot(x[i,:]) + B.dot(u[np.newaxis,i])
    return x

x0 = np.array([4,7])
u = np.array([-4, -2, -1, -0.5, 0, 0.5, 1, 3, 5])
n = len(u)+1
x = advance_linear_system(x0,u,n)
X1 = x[:-1,:]
X2 = x[1:,:]
C = u[:,np.newaxis]
print('X1 = ', X1)
print('X2 = ', X2)
print('C = ', C)

Apply DMD on the data from the controlled system

In [ ]:
U, s, Vh = np.linalg.svd(X1.T, full_matrices=False)
print('U = ', U)
print('s = ', s)
print('V = ', Vh.T)

A = np.dot(X2.T,np.dot(Vh.T*(s**(-1)),U.T))
print('A = ', A)

This is obviously not correct.
So let's apply DMDc on the data from the controlled system with known B

In [ ]:
U, s, Vh = np.linalg.svd(X1.T, full_matrices=False)
A = np.dot(X2.T-np.dot(B,C.T),np.dot(Vh.T*(s**(-1)),U.T))
print('A = ', A)

This yields the correct system matrix. Let's further assume B is also unknown.

In [ ]:
r = 3
Omega = np.vstack([X1.T,C.T])
U, s, Vh = np.linalg.svd(Omega, full_matrices=False)
Ur = U[:,0:r]
sr = s[0:r]
Vr = Vh[0:r,:].T
G = np.dot(X2.T,np.dot(Vr*(sr**(-1)),Ur.T))
print('G = ', G)

Aest = G[:,0:2]
Best = G[:,2]

np.allclose(A,Aest)
np.allclose(B,Best[:,np.newaxis])


In [ ]:
DMDc = pk.DMDc(svd_rank=3)

model = pk.Koopman(regressor=DMDc)
model.fit(x,C)
Aest = model.state_transition_matrix
Best = model.control_matrix

np.allclose(A,Aest)
np.allclose(B,Best)